In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Set Up Environment

In [12]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20201018044700.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://13.57.193.25:9000/",        "nid": 3}}

env = connections[network]

In [13]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [14]:
contracts


{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxcf9ba18f027f83d0d2fc5a872eb9c5176c554858'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxafaf7e01a85efdcb0c6e408f84b6f11e06275657'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx01e69f405052eebe0d0b040cc2dc648e5183d700'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxa7e986d1e18d3d93f246d84b6745e0b1b254e819'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cxa7e986d1e18d3d93f246d84b6745e0b1b254e819'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxad20839f2d321651506227b9571c65fa164f69c5'}}

In [4]:
contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxcf9ba18f027f83d0d2fc5a872eb9c5176c554858'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxafaf7e01a85efdcb0c6e408f84b6f11e06275657'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx01e69f405052eebe0d0b040cc2dc648e5183d700'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxa7e986d1e18d3d93f246d84b6745e0b1b254e819'},
 'dummy_oracle': {'zip': 'token_contracts/dummy_oracle.zip',
  'SCORE': 'cxa7e986d1e18d3d93f246d84b6745e0b1b254e819'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxad20839f2d321651506227b9571c65fa164f69c5'}}

In [ ]:
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cxcc4688bda4952366d7ed2a97deed2fcb8cbbf1b9'}

In [ ]:
testnet_contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx71698f1976085e9993d6d9a5d8362adfc091c63a'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxed5cc05b631f441d300d80d3b51ec63800962e3f'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx137bfc83d24f04cac374bb5088505184f2dfb87e'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxc8ddb41541b9479a38b741fc32679e9a73530312'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cx59ed8122422ae676904ea2f85855bcecccc73aba'}}

In [7]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [9]:
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

In [17]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
deploy = ['loans', 'staking', 'sicx', 'icd', 'dummy_oracle', 'bal']
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

saving contracts list as contracts_20201018051644.pkl


{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxcf9ba18f027f83d0d2fc5a872eb9c5176c554858'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxafaf7e01a85efdcb0c6e408f84b6f11e06275657'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx01e69f405052eebe0d0b040cc2dc648e5183d700'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxa7e986d1e18d3d93f246d84b6745e0b1b254e819'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip', 'SCORE': ''},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxad20839f2d321651506227b9571c65fa164f69c5'}}

In [425]:
api = icon_service.get_score_api(contracts['loans']['SCORE'])

In [426]:
api

[{'type': 'function',
  'name': 'add_asset',
  'inputs': [{'name': '_token_address', 'type': 'Address'}],
  'outputs': []},
 {'type': 'function',
  'name': 'get_account_positions',
  'inputs': [{'name': '_owner', 'type': 'Address'}],
  'outputs': [{'type': 'dict'}],
  'readonly': '0x1'},
 {'type': 'function',
  'name': 'get_available_assets',
  'inputs': [],
  'outputs': [{'type': 'list'}],
  'readonly': '0x1'},
 {'type': 'function',
  'name': 'get_oracle_address',
  'inputs': [],
  'outputs': [{'type': 'Address'}],
  'readonly': '0x1'},
 {'type': 'function',
  'name': 'get_price_in_icx',
  'inputs': [{'name': '_symbol', 'type': 'str'}],
  'outputs': []},
 {'type': 'function',
  'name': 'get_sicx_address',
  'inputs': [],
  'outputs': [{'type': 'Address'}],
  'readonly': '0x1'},
 {'type': 'function',
  'name': 'get_sicx_value',
  'inputs': [{'name': '_amount', 'type': 'int'}],
  'outputs': []},
 {'type': 'function',
  'name': 'get_staking_contract',
  'inputs': [],
  'outputs': [{'type

In [530]:
module = ("from iconsdk.icon_service import IconService\n"
        + "from iconsdk.providers.http_provider import HTTPProvider\n"
        + "from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder\n"
        + "from iconsdk.builder.call_builder import CallBuilder\n"
        + "from iconsdk.signed_transaction import SignedTransaction\n"
        + "icon_service = IconService(HTTPProvider('http://13.57.193.25:9000/', 3))\n"
        + "\n"
        + "class Test(object):\n"
        + "\n"
        + "    def __init__(self):\n"
        + "        pass\n"
        + "\n")
filename = 'methods.py'
for item in api:
    if item.get('readonly', '') == '0x1' and item['type'] == 'function':
        func_params = [f"{param['name']}" for param in item['inputs']]
        params = f'{{{(f"{param}": param) for param in func_params}}}'
        module += (f"    def {item['name']}({', '.join(['self'] + func_params)}):\n"
                 + f"        call = CallBuilder().from_('{wallet.get_address()}')\\\n"
                 + f"            .to('{contracts['loans']['SCORE']}')\\\n"
                 + f"            .method('{item['name']}')\\\n"
                 + f"            .params({params})\\\n"
                 + f"            .build()\n"
                 + f"        return icon_service.call(call)\n"
                 + f"\n")
with open(filename, 'w') as f:
    f.write(module)

SyntaxError: invalid syntax (<fstring>, line 1)

In [510]:
from methods import Test

In [517]:
trial = Test()

In [558]:
func_params = ['one', 'two', 'three']

In [561]:
func_params = ['1', '2', '3']

In [535]:
params = f'{{{param: param for param in func_params}}}'

NameError: name 'param' is not defined

In [557]:
one = 1
two = 2
three = 3

In [562]:
"{" + ', '.join(['\"' + param + f'\": {param}' for param in func_params]) + "}"

'{"1": 1, "2": 2, "3": 3}'

In [563]:
json.loads("{" + ', '.join(['\"' + param + f'\": {param}' for param in func_params]) + "}")

{'1': 1, '2': 2, '3': 3}

In [519]:
trial.name()

TypeError: params() missing 1 required positional argument: 'params'

In [515]:
class Test(object):

    def __init__(self):
        pass

    def get_account_positions(self, _owner):
        call = CallBuilder().from_('hxe7af5fcfd8dfc67530a01a0e403882687528dfcb')\
            .to('cx6927160050f452ae32163c895c08e0171aa6073c')\
            .method('get_account_positions')\
            .params(_owner)\
            .build()
        return icon_service.call(call)

    def get_available_assets(self):
        call = CallBuilder().from_('hxe7af5fcfd8dfc67530a01a0e403882687528dfcb')\
            .to('cx6927160050f452ae32163c895c08e0171aa6073c')\
            .method('get_available_assets')\
            .params()\
            .build()
        return icon_service.call(call)

    def get_oracle_address(self):
        call = CallBuilder().from_('hxe7af5fcfd8dfc67530a01a0e403882687528dfcb')\
            .to('cx6927160050f452ae32163c895c08e0171aa6073c')\
            .method('get_oracle_address')\
            .params()\
            .build()
        return icon_service.call(call)

    def get_sicx_address(self):
        call = CallBuilder().from_('hxe7af5fcfd8dfc67530a01a0e403882687528dfcb')\
            .to('cx6927160050f452ae32163c895c08e0171aa6073c')\
            .method('get_sicx_address')\
            .params()\
            .build()
        return icon_service.call(call)

    def get_staking_contract(self):
        call = CallBuilder().from_('hxe7af5fcfd8dfc67530a01a0e403882687528dfcb')\
            .to('cx6927160050f452ae32163c895c08e0171aa6073c')\
            .method('get_staking_contract')\
            .params()\
            .build()
        return icon_service.call(call)

    def name(self):
        call = CallBuilder().from_('hxe7af5fcfd8dfc67530a01a0e403882687528dfcb')\
            .to('cx6927160050f452ae32163c895c08e0171aa6073c')\
            .method('name')\
            .params()\
            .build()
        return icon_service.call(call)

## Deploy All SCOREs

In [18]:
# Deploy a single SCORE
score = 'dummy_oracle'
loc = {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': ''}
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(loc["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
contracts[score]["SCORE"] = res.get('scoreAddress', '')


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


In [19]:
res

{'txHash': '0x06dc47a25a41191527239db57b0a75837aab82441ff05b0bb17ed531273e96b3',
 'blockHeight': 94543,
 'blockHash': '0xc798a34276dfd0a85b8ba6755333bc9a3c020b37974eba1ec6463baaa17d526b',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'stepUsed': 1759045600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1759045600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [138]:
step_limit = 4000100000

for score, loc in contracts.items():
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}


{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}


...

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}


{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx71698f1976085e9993d6d9a5d8362adfc091c63a'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxed5cc05b631f441d300d80d3b51ec63800962e3f'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx137bfc83d24f04cac374bb5088505184f2dfb87e'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxc8ddb41541b9479a38b741fc32679e9a73530312'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cx59ed8122422ae676904ea2f85855bcecccc73aba'}}

## Update a SCORE

In [114]:
contract = 'loans'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x27781a576713cc938fd30d7e397bdee18027db2ed7dcfd0c102148b010be00ca',
 'blockHeight': 20881,
 'blockHash': '0xb8f037b5ac6862c11cf677a113f2b506a7cec4d652c8abeec256a95d71f34a31',
 'txIndex': 1,
 'to': 'cxcf9ba18f027f83d0d2fc5a872eb9c5176c554858',
 'scoreAddress': 'cxcf9ba18f027f83d0d2fc5a872eb9c5176c554858',
 'stepUsed': 2283805600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2283805600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Set parameters

In [140]:
settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'icd', 'method': 'setAdmin', 'params':{'_admin': contracts['loans']['SCORE']}},
            {'contract': 'bal', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'bal', 'method': 'mint', 'params':{'_amount': 10 * ICX}},
            {'contract': 'staking', 'method': 'set_sICX_address', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'set_sicx_address', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['icd']['SCORE']}},
            {'contract': 'loans', 'method': 'set_staking_contract', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'loans', 'method': 'toggle_loans_on', 'params':{}}]

In [141]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setAdmin, with parameters {'_admin': 'cxed5cc05b631f441d300d80d3b51ec63800962e3f'} on the sicx contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}


{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xb02603dad32a806f48ccf77cbdfecc26e6eb60c2264b791b639fb77c2283dc40', 'blockHeight': 8171840, 'blockHash': '0x570cc1ecafad86127afdc4b45889626f0d869abb3dc280eaaa06f6b1273faf6a', 'txIndex': 1, 'to': 'cx137bfc83d24f04cac374bb5088505184f2dfb87e', 'stepUsed': 148920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 148920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xd59f1f1b0f0c664c9a93034d55fb62b87337375f5c1ec64014b0ed0ac4e80169', 'blockHeight': 8171855, 'blockHash': '0x0420d75594ca3b56d21c11ee1bde11925da3573f687970b0e79b31f48d54ebfd', 'txIndex': 1, 'to': 'cx71698f1976085e9993d6d9a5d8362adfc091c63a', 'stepUsed': 151720, 'stepPrice': 10000000000, 'cumulativeStepUsed': 151720, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x227df8d909290d7af076eafe65955c0538bbf1e21bd4ceb09bc35acc20a1d324', 'blockHeight': 8171857, 'blockHash': '0x62fdf6041962e2644a815b2b1796d895200e04ee0e233c04d324333f24925fa8', 'txIndex': 1, 'to': 'cx71698f1976085e9993d6d9a5d8362adfc091c63a', 'stepUsed': 130680, 'stepPrice': 10000000000, 'cumulativeStepUsed': 130680, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Run through read-only methods on all SCOREs

### A single read-only method

In [102]:
params = {'_owner': wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('get_account_positions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'ICD': '0x410d586a20a4c00000'}

In [146]:
ro_methods = {"loans": ["name", "get_oracle_address", "get_staking_contract", "get_sicx_address", "get_available_assets", "get_total_collateral"],
              "staking": ["name", "get_sICX_address", "get_rate"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "bal": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [147]:
results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        if method in ['totalSupply', 'decimals', 'get_total_collateral', 'get_rate']:
            result = int(result, 0)
            if method in ['totalSupply', 'get_total_collateral', 'get_rate']:
                result = result / ICX
        print(f'{method}: {result}')
        results[score][method] = result


SCORE: loans
name: BalancedLoans
get_oracle_address: cx61a36e5d10412e03c907a507d1e8c6c3856d9964
get_staking_contract: cxed5cc05b631f441d300d80d3b51ec63800962e3f
get_sicx_address: cx137bfc83d24f04cac374bb5088505184f2dfb87e
get_available_assets: [['ICD', 'cxc8ddb41541b9479a38b741fc32679e9a73530312', 'USD']]
get_total_collateral: 825.0

SCORE: staking
name: Staking
get_sICX_address: cx137bfc83d24f04cac374bb5088505184f2dfb87e
get_rate: 1.0

SCORE: sicx
name: StakedICX
symbol: sICX
decimals: 18
totalSupply: 830.0
getAdmin: cxed5cc05b631f441d300d80d3b51ec63800962e3f

SCORE: icd
name: ICONDollar
symbol: ICD
decimals: 18
totalSupply: 35.0
getAdmin: cx71698f1976085e9993d6d9a5d8362adfc091c63a

SCORE: bal
name: BalanceToken
symbol: BAL
decimals: 18
totalSupply: 10.0
getAdmin: hx3f01840a599da07b0f620eeae7aa9c574169a4be


## Depost ICX and Mint ICD Loan

In [142]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_for\": \"hx3f01840a599da07b0f620eeae7aa9c574169a4be\", \"_asset\": \"ICD\", \"_amount\": 40000000000000000000}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}


{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 4 seconds...
{'txHash': '0x986985a713eb9085ef94cf38c57af447c16ede3813aae97865995745005d430f', 'blockHeight': 8171958, 'blockHash': '0x36b06ff454c0086694643bb2a187f7e760a3292fe55799ce47eb93952b1b64c5', 'txIndex': 1, 'to': 'cxed5cc05b631f441d300d80d3b51ec63800962e3f', 'stepUsed': 484900, 'stepPrice': 10000000000, 'cumulativeStepUsed': 484900, 'eventLogs': [{'scoreAddress': 'cxc8ddb41541b9479a38b741fc32679e9a73530312', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc8ddb41541b9479a38b741fc32679e9a73530312', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x22b1c8c1227a00000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cxc8ddb41541b9479a38b741fc32679e9a73530312', 'indexed': ['Mint(Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x22b1c8c1227a00000', '0x4e6f6e65']}, {'scoreAddress': 'cx71698f1976085e9993d6d9a5d8362adfc091c63a', 'indexed': ['OriginateLoan(Address,str,int,str)

### Just deposit collateral

In [143]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_for\": \"hx3f01840a599da07b0f620eeae7aa9c574169a4be\", \"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(30 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}


{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 4 seconds...
{'txHash': '0x370d36cefe5fffa5ba87ffd1f22b66d8e23ed9de612436124808103be0a81c4e', 'blockHeight': 8171979, 'blockHash': '0x4d8598c5abb28374655d8303f32749ee65ff4b325a2ff0b50929fd3ff9e85000', 'txIndex': 1, 'to': 'cxed5cc05b631f441d300d80d3b51ec63800962e3f', 'stepUsed': 301380, 'stepPrice': 10000000000, 'cumulativeStepUsed': 301380, 'eventLogs': [{'scoreAddress': 'cx137bfc83d24f04cac374bb5088505184f2dfb87e', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx137bfc83d24f04cac374bb5088505184f2dfb87e', 'cx71698f1976085e9993d6d9a5d8362adfc091c63a', '0x1a055690d9db80000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f666f72223a2022687833663031383430613539396461303762306636323065656165376161396335373431363961346265222c20225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']}, {'scoreAddress': 'cx137bfc83d24f04cac374bb5088505184f2dfb87e'

### Repay Some ICD

In [144]:
data = "{\"method\": \"_repay_loan\", \"params\": {}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}


{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x48eef94e53224e280e7b4029d58a157b64bfbbbe4bc04088c22e13dc6f60cbb1', 'blockHeight': 8171994, 'blockHash': '0xfd5845f133d701559676a4707a6ed70c34f23877e56bf6bd66eebc01bf109589', 'txIndex': 1, 'to': 'cxc8ddb41541b9479a38b741fc32679e9a73530312', 'stepUsed': 272400, 'stepPrice': 10000000000, 'cumulativeStepUsed': 272400, 'eventLogs': [{'scoreAddress': 'cxc8ddb41541b9479a38b741fc32679e9a73530312', 'indexed': ['Burn(Address,int)', 'cx71698f1976085e9993d6d9a5d8362adfc091c63a'], 'data': ['0x4563918244f40000']}, {'scoreAddress': 'cx71698f1976085e9993d6d9a5d8362adfc091c63a', 'indexed': ['LoanRepaid(Address,str,int,str)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'ICD', '0x4563918244f40000'], 'data': ['Loan of 5.0 ICD repaid to Balanced.']}, {'scoreAddress': 'cxc8ddb41541b9479a38b741fc32679e9a73530312', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574

### Withdraw Some Collateral

In [145]:
params = {'_value': 5 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdraw_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}


{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 4 seconds...
{'txHash': '0x0e61ba27e9207f1fc653a05adae833cd4b0ad5257f7ee6b773e5f3e581f3f442', 'blockHeight': 8172003, 'blockHash': '0xc5ad6647904b862cc059fa166103a4eaa0f0160f68f5d0c926bf1c900d9a8243', 'txIndex': 1, 'to': 'cx71698f1976085e9993d6d9a5d8362adfc091c63a', 'stepUsed': 241280, 'stepPrice': 10000000000, 'cumulativeStepUsed': 241280, 'eventLogs': [{'scoreAddress': 'cx137bfc83d24f04cac374bb5088505184f2dfb87e', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx71698f1976085e9993d6d9a5d8362adfc091c63a', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x4563918244f40000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cx71698f1976085e9993d6d9a5d8362adfc091c63a', 'indexed': ['TokenTransfer(Address,int,str)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x4563918244f40000'], 'data': ['Collateral withdrawn. 5000000000000000000 sICX sent to hx3f01840a599da07b0f620eeae7aa9c574169a4be.']}], 'logsBloom': b'\